In [10]:
import pandas as pd
import numpy as np

In [11]:
data = {
    "movie_id":[1,2,3,4,5],
    "title":["Inception","Intersteller","Avatar","Titanic","The Dark Knight"],
    "genre":["Sci-Fi|Action|Thriller","Drama|Sci-Fi","Action|Sci-Fi|Adventure","Romance|Drama","Action|Drama|Crime"]
}

In [12]:
movies = pd.DataFrame(data)

In [13]:
movies

,movie_id,title,genre
0,1,Inception,Sci-Fi|Action|Thriller
1,2,Intersteller,Drama|Sci-Fi
2,3,Avatar,Action|Sci-Fi|Adventure
3,4,Titanic,Romance|Drama
4,5,The Dark Knight,Action|Drama|Crime


In [14]:
ratings_data = {
    "user_id":[1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4],
    "movie_id":[2,3,4,1,1,2,5,4,3,4,2,5,4,1,2,5],
    "rating":[4,5,3,2,2,5,2,4,5,3,1,3,4,3,5,4]
}

In [15]:
ratings = pd.DataFrame(ratings_data)

In [16]:
ratings

,user_id,movie_id,rating
0,1,2,4
1,1,3,5
2,1,4,3
3,1,1,2
4,2,1,2
5,2,2,5
6,2,5,2
7,2,4,4
8,3,3,5
9,3,4,3


In [17]:
# Content-Based

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
vectorizer = CountVectorizer( tokenizer = lambda x:x.split('|') )

In [20]:
genre_matrix = vectorizer.fit_transform(movies['genre'])

C:\Users\Croma Campus\AppData\Roaming\Python\Python313\site-packages\sklearn\feature_extraction\text.py:526: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [21]:
genre_matrix

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 13 stored elements and shape (5, 7)>

In [22]:
cos_sim = cosine_similarity(genre_matrix)

In [23]:
cos_sim

array([[1.        , 0.40824829, 0.66666667, 0.        , 0.33333333],
       [0.40824829, 1.        , 0.40824829, 0.5       , 0.40824829],
       [0.66666667, 0.40824829, 1.        , 0.        , 0.33333333],
       [0.        , 0.5       , 0.        , 1.        , 0.40824829],
       [0.33333333, 0.40824829, 0.33333333, 0.40824829, 1.        ]])

In [24]:
genre_matrix.toarray()

array([[1, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 0, 1, 0],
       [1, 1, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 1, 0, 0],
       [1, 0, 1, 1, 0, 0, 0]])

In [25]:
vectorizer.get_feature_names_out()

array(['action', 'adventure', 'crime', 'drama', 'romance', 'sci-fi',
       'thriller'], dtype=object)

In [26]:
genre = pd.DataFrame(
    genre_matrix.toarray() , 
    columns = vectorizer.get_feature_names_out() , 
    index = movies['title']
)

In [27]:
genre

,action,adventure,crime,drama,romance,sci-fi,thriller
title,,,,,,,
Inception,1,0,0,0,0,1,1
Intersteller,0,0,0,1,0,1,0
Avatar,1,1,0,0,0,1,0
Titanic,0,0,0,1,1,0,0
The Dark Knight,1,0,1,1,0,0,0


In [28]:
cos = pd.DataFrame(
    cos_sim , 
    columns = movies['title'],
    index = movies['title']
)

In [29]:
cos

title,Inception,Intersteller,Avatar,Titanic,The Dark Knight
title,,,,,
Inception,1.000000,0.408248,0.666667,0.000000,0.333333
Intersteller,0.408248,1.000000,0.408248,0.500000,0.408248
Avatar,0.666667,0.408248,1.000000,0.000000,0.333333
Titanic,0.000000,0.500000,0.000000,1.000000,0.408248
The Dark Knight,0.333333,0.408248,0.333333,0.408248,1.000000


In [54]:
def recommend_by_genre(movie_title , top_n = 2):
    idx = movies[movies['title']==movie_title].index[0]
    sim_score = list( enumerate( cos_sim[idx] ) )
    sim_score = sorted(sim_score , key=lambda x:x[1] , reverse=True) 
    sim_score = sim_score[1:top_n+1]
    return movies.iloc[ [i[0] for i in sim_score] ][['title','genre']]

In [55]:
recommend_by_genre('Avatar')

,title,genre
0,Inception,Sci-Fi|Action|Thriller
1,Intersteller,Drama|Sci-Fi
